In [2]:
import numpy as np
import pandas as pd
import cvxpy as cp

In [3]:
rng = np.random.default_rng(42)

In [4]:
# Number of Riders
num_riders = 1

# Number of drivers
num_drivers = rng.poisson(lam=20)

# Goal is to route DRIVER to RIDER
# Equivalently, to match a rider with a driver

# Physical measure of distance between riders and drivers
distances = pd.DataFrame(rng.uniform(0, 10, (num_riders, num_drivers)),
                         columns=[f'Driver {j}' for j in range(1, num_drivers+1)],
                         index=[f'Rider {i}' for i in range(1,num_riders+1)])

# Map distances to a binary constraint which consists 1 if within MDR and 0 otherwise
# A value of 1 means the driver and rider can be connected.
MDR = 5 # don't dispatch outside of MDR miles
maximum_dispatch_radius = (distances < MDR).apply(lambda x: np.where(x, 1, 0)) # np.where(distances < MDR, 1, 0)

# Temporal measure of distance between riders and drivers
# Expected Time of Arrival
# ETA should be a function of the distance, driver's allowable speed, and some idiosyncratic noise
# (need a more realistic way to assign a variance, maybe as a function of distance)
eta = distances.apply(lambda x: rng.pareto(x))# rng.exponential(x)) 

In [5]:
eta

,Driver 1,Driver 2,Driver 3,Driver 4,Driver 5,Driver 6,Driver 7,Driver 8,Driver 9,Driver 10,...,Driver 15,Driver 16,Driver 17,Driver 18,Driver 19,Driver 20,Driver 21,Driver 22,Driver 23,Driver 24
Rider 1,0.054216,0.011114,0.210143,0.07277,0.052392,0.174482,0.738491,0.054248,0.132412,0.071678,...,0.271868,7.841425,0.139952,0.011628,0.161365,0.465232,0.122537,0.031853,0.042093,0.088881


In [6]:
# distances = distances.transform(lambda x: x+20) # Sanity Check
distances

,Driver 1,Driver 2,Driver 3,Driver 4,Driver 5,Driver 6,Driver 7,Driver 8,Driver 9,Driver 10,...,Driver 15,Driver 16,Driver 17,Driver 18,Driver 19,Driver 20,Driver 21,Driver 22,Driver 23,Driver 24
Rider 1,8.585979,6.97368,0.941773,9.756224,7.611397,7.860643,1.281136,4.503859,3.70798,9.26765,...,5.545848,0.638173,8.276312,6.316644,7.580877,3.54526,9.70698,8.931211,7.783835,1.946387


In [7]:
maximum_dispatch_radius

,Driver 1,Driver 2,Driver 3,Driver 4,Driver 5,Driver 6,Driver 7,Driver 8,Driver 9,Driver 10,...,Driver 15,Driver 16,Driver 17,Driver 18,Driver 19,Driver 20,Driver 21,Driver 22,Driver 23,Driver 24
Rider 1,0,0,1,0,0,0,1,1,1,0,...,0,1,0,0,0,1,0,0,0,1


<h3>Decision Variables</h3>


$$ \begin {align}
& \text {Decision variables in this questions would be pair/matches made, i.e. 1 if paired and 0 if not} \\
& \text {That is variable with the same shape as eta/distance: } \\  
& match = cp.Variable(eta.shape) \ \text { Where } match \in {[0,1]} \\\\
& \ \text {There might be other ways,} \\
& \text { but I will also specify that it is integer though since it can be 0 or 1:}\\
& \ match = cp.Variable(eta.shape, integer=True) \\ \\
\end {align}$$

<h3>Objective</h3>
$$ \begin {align} & \text {We need to implement first dispatch, and since it is optimization problem,}\\
& \text {Objective is finding match which brings least ETA for rider to be picked up satisfying mdr constraint} \\
& \text {Mimimize: }  g(match) = \sum_i match_i \times eta_i 
\end {align} $$

<h3>Constraints</h3>

$$ \begin {align} 
& \text {First is MDR constraint:   }  match_i \leq MDR_i \\
& \text {Second, I would add that there can be only one match: }  \sum_i match_i = 1 \\
& \text {We are not ranking and if driver doesn't accept the order, he/she will be excluded from a rider-driver relationship} \\
\end {align} $$

<h3>Solution in terms of a dispatching recommendation</h3>



In [8]:
#Version 1
#since there is only one rider we can specify size of variable as number of riders
match=cp.Variable(distances.shape[1],integer=True)
# our objective function is minimizing ETA, and since MDR is set of zeros and ones by multiplying
# it by ETA we will no longer need to add MDR as a constraint
# and matrix mulitplication works here too
obj=cp.Minimize(match @ (eta.T * maximum_dispatch_radius.T) )
constraint = [match>=0, match <=1,match @ (eta.T * maximum_dispatch_radius.T)>=min(eta.iloc[0,:])]
prob=cp.Problem(obj,constraint)

#solving the problem
prob.solve()


0.05424831544228037

In [9]:
df=pd.DataFrame(match.value).T
# df.index=df.index+1
# df.columns = pd.RangeIndex(1, len(df.columns)+1) 
# #since we used matrix multiplication
# df.T
df.columns = [f'Driver {j}' for j in range(1, match.shape[0]+1)]
df.index=['Rider 1']
df

,Driver 1,Driver 2,Driver 3,Driver 4,Driver 5,Driver 6,Driver 7,Driver 8,Driver 9,Driver 10,...,Driver 15,Driver 16,Driver 17,Driver 18,Driver 19,Driver 20,Driver 21,Driver 22,Driver 23,Driver 24
Rider 1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [10]:
# Version 2
# decision variable is the same size as ETA
match=cp.Variable(eta.shape,integer=True)
# since we need to use multiplication by position we can use cp.multiply and we are minimizing ETA cp.sum
obj=cp.Minimize(cp.sum(cp.multiply(match, eta)))
#there is match or no match
constraint=[match>=0, match<=1]

counter=0
for i in range(eta.shape[1]):
    # adding MDR constraint
    constraint.append(match[0,i]<=maximum_dispatch_radius.iloc[0,i])
    #adding constraint that there is only one match possible
    counter+=match[0,i]
constraint.append(counter ==1)
prob=cp.Problem(obj, constraint)
prob.solve()

0.05424831544228037

In [11]:
df = pd.DataFrame(match.value,
                         columns=[f'Driver {j}' for j in range(1, match.shape[1]+1)],
                         index=[f'Rider {i}' for i in range(1,match.shape[0]+1)])
df

,Driver 1,Driver 2,Driver 3,Driver 4,Driver 5,Driver 6,Driver 7,Driver 8,Driver 9,Driver 10,...,Driver 15,Driver 16,Driver 17,Driver 18,Driver 19,Driver 20,Driver 21,Driver 22,Driver 23,Driver 24
Rider 1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


<h3>If we Implement matching of many to many</h3>

In [12]:
rng = np.random.default_rng(42)

In [13]:
# Number of Riders
num_riders = rng.poisson(lam=15)

# Number of drivers
num_drivers = rng.poisson(lam=15)

# Goal is to route DRIVER to RIDER
# Equivalently, to match a rider with a driver

# Physical measure of distance between riders and drivers
distances = pd.DataFrame(rng.uniform(0, 30, (num_riders, num_drivers)),
                         columns=[f'Driver {j}' for j in range(1, num_drivers+1)],
                         index=[f'Rider {i}' for i in range(1,num_riders+1)])

# Temporal measure of distance between riders and drivers
# Expected Time of Arrival
# ETA should be a function of the distance, driver's allowable speed, and some idiosyncratic noise
# (need a more realistic way to assign a variance, maybe as a function of distance)
eta = distances.apply(lambda x: rng.pareto(x)) 


In [14]:
eta

,Driver 1,Driver 2,Driver 3,Driver 4,Driver 5,Driver 6,Driver 7,Driver 8,Driver 9,Driver 10,Driver 11,Driver 12,Driver 13,Driver 14,Driver 15,Driver 16,Driver 17,Driver 18
Rider 1,0.063526,0.486780,0.012106,0.027373,0.022479,0.014494,0.118065,6.075330e-01,0.067738,0.944572,0.024951,0.011973,0.013173,0.308573,0.028032,0.169770,0.109341,0.560946
Rider 2,0.248347,0.877538,0.227651,0.015941,0.065569,0.049331,0.019989,3.700551e-02,0.040100,0.200043,0.174144,0.023295,0.310061,0.197105,0.133713,0.032297,0.001956,0.167547
Rider 3,0.001538,0.012287,0.251564,0.037826,0.000859,0.020208,0.218212,4.846250e+06,0.041857,0.121579,0.001143,0.018190,0.098644,0.079491,0.212200,0.530337,0.092002,0.184825
Rider 4,0.067861,0.025888,0.027469,0.067392,0.111707,0.186770,0.672072,2.396151e-02,0.132433,0.015500,0.090009,0.037363,0.114589,0.259361,0.183641,0.045809,0.008773,0.007093
Rider 5,0.004850,0.107339,0.020358,1.008363,0.031519,1.087487,0.004370,3.867961e-02,0.011159,0.071111,1.429397,0.028954,0.045596,0.046097,0.077539,0.039752,0.007066,0.705123
Rider 6,1.001942,0.035147,0.024661,0.009173,0.127556,0.079233,0.060852,5.473993e-02,0.020133,0.054029,0.267101,0.013120,0.277899,0.024043,0.172007,0.033663,0.238994,0.016490
Rider 7,0.003812,0.101547,0.016014,0.065231,0.003376,0.122381,0.051385,5.924328e-03,0.230812,0.049417,0.042624,0.138119,0.293317,0.048901,0.148144,0.000879,0.035532,0.339986
Rider 8,0.091473,26.841782,0.006282,0.198894,0.013723,0.940601,0.340011,1.959335e-01,0.002736,0.273332,0.173345,0.084653,0.153671,0.180374,0.080745,427.269935,0.065623,0.025025
Rider 9,0.043976,0.058538,1.595765,0.000632,0.071684,0.004295,0.030670,1.964556e-02,0.152024,0.019887,0.080725,0.273153,0.160231,0.014484,0.080289,0.043899,0.410271,0.034298
Rider 10,0.682454,0.107257,0.367071,0.019731,0.553485,0.843472,0.027580,8.747272e-03,0.002220,0.053902,0.037098,0.036860,0.038825,0.047305,0.208807,0.149172,0.050306,0.121838


In [15]:
distances

,Driver 1,Driver 2,Driver 3,Driver 4,Driver 5,Driver 6,Driver 7,Driver 8,Driver 9,Driver 10,Driver 11,Driver 12,Driver 13,Driver 14,Driver 15,Driver 16,Driver 17,Driver 18
Rider 1,3.843409,13.511578,11.123941,27.802950,19.315954,24.682848,13.302426,6.817162,16.637544,1.914518,24.828935,18.949932,22.742632,10.635779,29.120941,26.793634,23.351505,5.839161
Rider 2,14.001630,1.314113,4.628685,20.491469,22.342865,29.025292,9.774761,11.113791,14.086674,5.684141,3.897645,14.271148,6.807280,20.094420,13.114558,24.980346,21.007953,9.370999
Rider 3,24.967794,24.142931,11.624351,8.649843,20.474865,4.192575,5.997246,0.220868,23.607731,19.945526,21.154961,23.421871,13.767473,17.062236,4.193910,3.435902,20.052089,14.132886
Rider 4,16.957083,22.949966,19.041550,16.607382,16.776215,9.118503,0.924535,13.101522,6.437540,12.255859,25.602092,7.018185,1.749082,8.441517,8.807813,19.857495,16.710965,23.516946
Rider 5,19.929406,12.191606,24.420612,5.009188,0.681362,2.701436,21.670781,13.856317,4.838153,15.031343,4.569363,20.889611,13.384688,11.430637,9.045363,18.908478,10.854378,2.629498
Rider 6,3.540177,28.856930,27.257421,20.991214,7.976099,29.075291,23.362527,21.506706,13.480845,8.167247,2.891729,27.078072,13.673289,6.070901,9.178699,17.376587,5.303183,25.698429
Rider 7,22.755586,21.583889,12.962791,18.819265,17.522939,19.495398,2.533330,12.474222,1.248425,14.819725,9.895836,4.335726,3.102089,17.629337,5.117789,27.753604,17.431834,10.406094
Rider 8,17.727465,0.684116,28.756776,14.469103,23.482057,2.481900,14.599750,14.721210,28.134794,17.151842,14.204682,8.009270,9.947070,15.620172,13.167344,0.648362,24.788758,26.884823
Rider 9,4.207473,16.621084,3.257272,20.167203,8.437014,19.782679,21.809838,23.059425,3.232228,27.480355,6.906420,1.122377,16.645574,11.127669,24.893692,24.247544,9.514167,28.586982
Rider 10,8.727535,15.451714,7.678953,28.081307,4.938235,1.347319,13.052912,29.771267,26.750318,22.458241,26.723775,26.803399,15.565751,9.477872,23.160373,19.849838,11.209732,2.834000


<h3>Decision Variables</h3>


$$ \begin {align}
& \text {Decision variables in this questions would be pair/matches made, i.e. 1 if paired and 0 if not} \\
& \text {That is variable with the same shape as eta or distance: } \\  
& match = cp.Variable(eta.shape) \text { or }  match = cp.Variable(distances.shape)\\
&\text { where } match \in {[0,1]} \\\\
& \text { I will also specify that it is integer since it can be 0 or 1:}\\
& \ match = cp.Variable(eta.shape, integer=True) \\ \\
\end {align}$$

<h3>Objective</h3>
$$ \begin {align} & \text {We need to implement Batching Dispatch Protocol which Minimizes the Aggregated En Route Time}\\
& \text {Objective is finding matches which bring least total ETA for rider to be picked up} \\
& \text {Mimimize: }  g(match) = \sum_i match_i \times eta_i 
\end {align} $$

<h3>Constraints</h3>

$$ \begin {align} 
& \text {First, I would add that there can be only one or no match for rider/driver: } \\
& \text {for riders: }\sum_{i \in riders} match[i,:] \leq 1  \text {for drivers: } \sum_{j \in drivers} match[:,j] \leq 1 \\
& \text {Second contraint is that match either 1 which means there is match or 0 no match: } \\
& match \leq 1 \text { and } match \geq 0 \\
& \text {Third contraint is total number of matches which depends on condition if there are more riders or drivers} \\
& \text {if } riders > drivers: \sum match = n_{drivers}  \text { and if } riders \leq drivers: \sum match = n_{riders} \\
& \text {From the questions it seems there is no MDR contraint, I won't add it :  } \\
& match_i \leq MDR_i \\
\end {align} $$

<h3>Solution in terms of a dispatching recommendation</h3>



In [16]:
# Variable the same size as ETA/distances
match=cp.Variable(distances.shape,integer=True)

#objective is minimizing ETA for batch as a whole, so we element-wise matrix multiplication and sum it 
obj= cp.sum(cp.multiply(match,  eta)) 

#counter for total matches which is equal to number of riders or drivers according to shape
total_matches=0

# there is match or no match constraint
constraint=[match>=0, match<=1]

# situation when number of riders is less that riders
if eta.shape[0]>eta.shape[1]:
    for i in range(eta.shape[0]):
        for j in range(eta.shape[1]):
            total_matches = total_matches + match[i,j]
# Total matches equal to number of drivers because there are less drivers than riders
    constraint.append(total_matches==match.shape[1])
    
    for l in range(match.shape[0]):
# constraint that 1 rider will match only with 1 driver 
        constraint.append(sum(match[l,:])<=1)
    for m in range(match.shape[1]):
# constraint that 1 driver will match only with 1 rider 
        constraint.append(sum(match[:,m])<=1)        

# when n_drivers > or = riders    
else:
    for i in range(eta.shape[0]):
        for j in range(eta.shape[1]):
            total_matches = total_matches + match[i,j]
# Total matches equal to number of riders, because there are less riders
    constraint.append(total_matches==match.shape[0])
    
    for l in range(match.shape[0]):
# constraint that 1 rider will match only with 1 driver 
        constraint.append(sum(match[l,:])<=1)
    for m in range(match.shape[1]):
# constraint that 1 driver will match only with 1 rider 
        constraint.append(sum(match[:,m])<=1)

# so objective is minimizing total en route time
objective=cp.Minimize(obj) 
prob=cp.Problem(objective, constraint)

prob.solve()

0.10146319689847205

In [17]:
df = pd.DataFrame(match.value,
                         columns=[f'Driver {j}' for j in range(1, match.shape[1]+1)],
                         index=[f'Rider {i}' for i in range(1,match.shape[0]+1)])
df

,Driver 1,Driver 2,Driver 3,Driver 4,Driver 5,Driver 6,Driver 7,Driver 8,Driver 9,Driver 10,Driver 11,Driver 12,Driver 13,Driver 14,Driver 15,Driver 16,Driver 17,Driver 18
Rider 1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
Rider 2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
Rider 3,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Rider 4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
Rider 5,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Rider 6,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
Rider 7,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
Rider 8,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Rider 9,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Rider 10,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


<h2>Batching Dispatch Protocol: Maximize Profit Model with a Maximum Dispatch Radius</h2>
<br>


In [18]:
rng = np.random.default_rng(42)

In [19]:
# Number of Riders
num_riders = rng.poisson(lam=15)

# Number of drivers
num_drivers = rng.poisson(lam=15)

# Goal is to route DRIVER to RIDER
# Equivalently, to match a rider with a driver

# Physical measure of distance between riders and drivers
distances = pd.DataFrame(rng.uniform(0, 30, (num_riders, num_drivers)),
                         columns=[f'Driver {j}' for j in range(1, num_drivers+1)],
                         index=[f'Rider {i}' for i in range(1,num_riders+1)])

# Map distances to a binary constraint which consists 1 if within MDR and 0 otherwise
# A value of 1 means the driver and rider can be connected.
MDR = 15 # don't dispatch outside of MDR miles
maximum_dispatch_radius = (distances < MDR).apply(lambda x: np.where(x, 1, 0)) # np.where(distances < MDR, 1, 0)

# Temporal measure of distance between riders and drivers
# Expected Time of Arrival
# ETA should be a function of the distance, driver's allowable speed, and some idiosyncratic noise
# need a more realistic way to assign a variance, maybe as a function of distance
profits = distances.apply(lambda x: x + rng.lognormal(mean=1, sigma=3)) 

In [20]:
profits

,Driver 1,Driver 2,Driver 3,Driver 4,Driver 5,Driver 6,Driver 7,Driver 8,Driver 9,Driver 10,Driver 11,Driver 12,Driver 13,Driver 14,Driver 15,Driver 16,Driver 17,Driver 18
Rider 1,32.262213,15.045836,102.386560,53.660612,659.617014,49.027790,13.326754,9.040791,16.718324,2.488682,277.903786,37.354547,23.076573,10.765658,32.120143,26.864307,23.714480,12.770330
Rider 2,42.420434,2.848371,95.891304,46.349131,662.643925,53.370233,9.799089,13.337420,14.167455,6.258305,256.972496,32.675762,7.141221,20.224299,16.113760,25.051020,21.370928,16.302168
Rider 3,53.386598,25.677189,102.886971,34.507505,660.775926,28.537516,6.021574,2.444497,23.688512,20.519690,274.229813,41.826486,14.101414,17.192115,7.193112,3.506576,20.415064,21.064055
Rider 4,45.375887,24.484224,110.304169,42.465044,657.077275,33.463444,0.948863,15.325151,6.518321,12.830024,278.676943,25.422799,2.083023,8.571396,11.807015,19.928169,17.073940,30.448115
Rider 5,48.348210,13.725864,115.683231,30.866850,640.982423,27.046377,21.695109,16.079946,4.918934,15.605508,257.644214,39.294226,13.718629,11.560516,12.044565,18.979152,11.217353,9.560666
Rider 6,31.958981,30.391188,118.520040,46.848876,648.277159,53.420232,23.386855,23.730335,13.561625,8.741411,255.966580,45.482687,14.007230,6.200780,12.177901,17.447261,5.666159,32.629597
Rider 7,51.174390,23.118147,104.225411,44.676927,657.824000,43.840339,2.557658,14.697851,1.329206,15.393889,262.970688,22.740340,3.436030,17.759216,8.116991,27.824277,17.794809,17.337263
Rider 8,46.146268,2.218374,120.019396,40.326765,663.783117,26.826841,14.624078,16.944839,28.215574,17.726006,267.279533,26.413885,10.281011,15.750051,16.166546,0.719036,25.151733,33.815992
Rider 9,32.626276,18.155342,94.519892,46.024865,648.738074,44.127620,21.834167,25.283054,3.313009,28.054520,259.981271,19.526991,16.979515,11.257548,27.892894,24.318218,9.877142,35.518150
Rider 10,37.146339,16.985972,98.941572,53.938969,645.239295,25.692260,13.077240,31.994896,26.831098,23.032405,279.798626,45.208014,15.899692,9.607751,26.159575,19.920512,11.572707,9.765169


In [21]:
distances

,Driver 1,Driver 2,Driver 3,Driver 4,Driver 5,Driver 6,Driver 7,Driver 8,Driver 9,Driver 10,Driver 11,Driver 12,Driver 13,Driver 14,Driver 15,Driver 16,Driver 17,Driver 18
Rider 1,3.843409,13.511578,11.123941,27.802950,19.315954,24.682848,13.302426,6.817162,16.637544,1.914518,24.828935,18.949932,22.742632,10.635779,29.120941,26.793634,23.351505,5.839161
Rider 2,14.001630,1.314113,4.628685,20.491469,22.342865,29.025292,9.774761,11.113791,14.086674,5.684141,3.897645,14.271148,6.807280,20.094420,13.114558,24.980346,21.007953,9.370999
Rider 3,24.967794,24.142931,11.624351,8.649843,20.474865,4.192575,5.997246,0.220868,23.607731,19.945526,21.154961,23.421871,13.767473,17.062236,4.193910,3.435902,20.052089,14.132886
Rider 4,16.957083,22.949966,19.041550,16.607382,16.776215,9.118503,0.924535,13.101522,6.437540,12.255859,25.602092,7.018185,1.749082,8.441517,8.807813,19.857495,16.710965,23.516946
Rider 5,19.929406,12.191606,24.420612,5.009188,0.681362,2.701436,21.670781,13.856317,4.838153,15.031343,4.569363,20.889611,13.384688,11.430637,9.045363,18.908478,10.854378,2.629498
Rider 6,3.540177,28.856930,27.257421,20.991214,7.976099,29.075291,23.362527,21.506706,13.480845,8.167247,2.891729,27.078072,13.673289,6.070901,9.178699,17.376587,5.303183,25.698429
Rider 7,22.755586,21.583889,12.962791,18.819265,17.522939,19.495398,2.533330,12.474222,1.248425,14.819725,9.895836,4.335726,3.102089,17.629337,5.117789,27.753604,17.431834,10.406094
Rider 8,17.727465,0.684116,28.756776,14.469103,23.482057,2.481900,14.599750,14.721210,28.134794,17.151842,14.204682,8.009270,9.947070,15.620172,13.167344,0.648362,24.788758,26.884823
Rider 9,4.207473,16.621084,3.257272,20.167203,8.437014,19.782679,21.809838,23.059425,3.232228,27.480355,6.906420,1.122377,16.645574,11.127669,24.893692,24.247544,9.514167,28.586982
Rider 10,8.727535,15.451714,7.678953,28.081307,4.938235,1.347319,13.052912,29.771267,26.750318,22.458241,26.723775,26.803399,15.565751,9.477872,23.160373,19.849838,11.209732,2.834000


In [22]:
maximum_dispatch_radius

,Driver 1,Driver 2,Driver 3,Driver 4,Driver 5,Driver 6,Driver 7,Driver 8,Driver 9,Driver 10,Driver 11,Driver 12,Driver 13,Driver 14,Driver 15,Driver 16,Driver 17,Driver 18
Rider 1,1,1,1,0,0,0,1,1,0,1,0,0,0,1,0,0,0,1
Rider 2,1,1,1,0,0,0,1,1,1,1,1,1,1,0,1,0,0,1
Rider 3,0,0,1,1,0,1,1,1,0,0,0,0,1,0,1,1,0,1
Rider 4,0,0,0,0,0,1,1,1,1,1,0,1,1,1,1,0,0,0
Rider 5,0,1,0,1,1,1,0,1,1,0,1,0,1,1,1,0,1,1
Rider 6,1,0,0,0,1,0,0,0,1,1,1,0,1,1,1,0,1,0
Rider 7,0,0,1,0,0,0,1,1,1,1,1,1,1,0,1,0,0,1
Rider 8,0,1,0,1,0,1,1,1,0,0,1,1,1,0,1,1,0,0
Rider 9,1,0,1,0,1,0,0,0,1,0,1,1,0,1,0,0,1,0
Rider 10,1,0,1,0,1,1,1,0,0,0,0,0,0,1,0,0,1,1


<h3>Decision Variables</h3>


$$ \begin {align}
& \text {Decision variables in this questions would be pair/matches made, i.e. 1 if paired and 0 if not} \\
& \text {That is variable with the same shape as profits or distance: } \\  
& match = cp.Variable(profits.shape) \text { or }  match = cp.Variable(distances.shape)\\
& \ \text { where } match \in {[0,1]}\\\\
& \text {I will also specify that it is integer, since it can be 0 or 1:}\\
& \ match = cp.Variable(profits.shape, integer=True) \\ \\
\end {align}$$

<h3>Objective</h3>
$$ \begin {align} & \text {We need to implement Batching Dispatch Protocol which Maximizes the total profits while satisfying the MDR constraints}\\
& \text {Objective is finding matches which bring the most profit:} \\
& \text {Maximize: }  g(match) = \sum_i match_i \times profits_i 
\end {align} $$

<h3>Constraints</h3>

$$ \begin {align} 
& \text {First matches should satisfy MDR contraint:  } \\
& match_i \leq MDR_i \\
& \text {Second contraint is that match either 1 which means there is match or 0 no match: } \\
& match \leq 1 \text { and } match \geq 0 \\
& \text {Third contraint is total number of matches which depends on condition if there are more riders or drivers} \\
& \text {if } riders > drivers: \sum match = n_{drivers}  \text { and if } riders \leq drivers: \sum match = n_{riders} \\
& \text {Fourth, I would add that there can be only one or no match for rider/driver: } \\
& \text {for riders: }\sum_{i \in riders} match[i,:] \leq 1  \text {for drivers: } \sum_{j \in drivers} match[:,j] \leq 1 \\
\end {align} $$

<h3>Solution in terms of a dispatching recommendation</h3>



In [23]:
# Variable the same size as ETA/distances
match=cp.Variable(profits.shape,integer=True)

#objective is minimizing ETA for batch as a whole
obj= cp.sum(cp.multiply(match,  (profits))) 
#counter for total matches which is equal to number of riders or drivers according to shape
total_matches=0
# there is match or no match constraint
constraint=[match>=0, match<=1]
# situation when number of riders is less that riders
if profits.shape[0]>profits.shape[1]:
    for k in range(profits.shape[1]):
        constraint.append(sum(cp.multiply(match[:,k],  profits.iloc[:,k]))>= min(profits.iloc[:,k]))
    for i in range(profits.shape[0]):
        for j in range(profits.shape[1]):
            
            constraint.append(match[i,j] <= maximum_dispatch_radius.iloc[i,j])
            total_matches = total_matches + match[i,j]
# Total matches equal to number of drivers because there are less drivers than riders
            
    constraint.append(total_matches==match.shape[1])
    for l in range(match.shape[0]):
# constraint that 1 rider will match only with 1 driver 
        constraint.append(sum(match[l,:])<=1)
    for m in range(match.shape[1]):
# constraint that 1 driver will match only with 1 rider 
        constraint.append(sum(match[:,m])<=1)        
else:
# when n_drivers > or = riders
    for i in range(profits.shape[0]):
        constraint.append(cp.sum(cp.multiply(match[i,:], profits.iloc[i,:]))>=min(profits.iloc[i,:]))
        for j in range(profits.shape[1]):
            constraint.append(match[i,j] <= maximum_dispatch_radius.iloc[i,j])
#             if maximum_dispatch_radius.iloc[i,j] == 0:
#                 constraint.append(match[i,j]==0)
#             else:
#                 constraint.append(match[i,j]<=1)
            total_matches = total_matches + match[i,j]
# Total matches equal to number of riders, because there are less riders
    constraint.append(total_matches==match.shape[0])
    for l in range(match.shape[0]):
# constraint that 1 rider will match only with 1 driver 
        constraint.append(sum(match[l,:])<=1)
    for m in range(match.shape[1]):
# constraint that 1 driver will match only with 1 rider 
        constraint.append(sum(match[:,m])<=1)

# so objective is minimizing total en route time
objective=cp.Maximize(obj) 
prob=cp.Problem(objective, constraint)

prob.solve()

1322.737816950058

In [24]:
df = pd.DataFrame(match.value,
                         columns=[f'Driver {j}' for j in range(1, match.shape[1]+1)],
                         index=[f'Rider {i}' for i in range(1,match.shape[0]+1)])
df

,Driver 1,Driver 2,Driver 3,Driver 4,Driver 5,Driver 6,Driver 7,Driver 8,Driver 9,Driver 10,Driver 11,Driver 12,Driver 13,Driver 14,Driver 15,Driver 16,Driver 17,Driver 18
Rider 1,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Rider 2,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Rider 3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
Rider 4,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Rider 5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
Rider 6,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Rider 7,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Rider 8,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Rider 9,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Rider 10,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
